In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
resnet152v2 = \
    model_development.resnet152v2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = resnet152v2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.8334 - accuracy: 0.6755 - precision: 0.6902 - recall: 0.5214
Epoch 1: val_accuracy improved from -inf to 0.52836, saving model to ../models/weights\ResNet152v2_1.00.00_fs.hdf5
119/119 [==============================] - 40s 258ms/step - loss: 0.8334 - accuracy: 0.6755 - precision: 0.6902 - recall: 0.5214 - val_loss: 1.2130 - val_accuracy: 0.5284 - val_precision: 0.5052 - val_recall: 0.9006
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5435 - accuracy: 0.7309 - precision: 0.7736 - recall: 0.5782
Epoch 2: val_accuracy did not improve from 0.52836
119/119 [==============================] - 31s 261ms/step - loss: 0.5435 - accuracy: 0.7309 - precision: 0.7736 - recall: 0.5782 - val_loss: 2.5201 - val_accuracy: 0.4806 - val_precision: 0.4806 - val_recall: 1.0000
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5010 - accuracy: 0.7430 - precision: 0.7866 - recall: 0.5979


In [6]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/ResNet152v2_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 48.3325 - accuracy: 0.5694 - precision: 0.5224 - recall: 0.6350
Epoch 1: val_accuracy improved from -inf to 0.58209, saving model to ../models/weights\ResNet152v2_1.00.00_ss.hdf5
119/119 [==============================] - 77s 543ms/step - loss: 48.3325 - accuracy: 0.5694 - precision: 0.5224 - recall: 0.6350 - val_loss: 3.6439 - val_accuracy: 0.5821 - val_precision: 0.5652 - val_recall: 0.5652
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.6663 - accuracy: 0.6966 - precision: 0.8396 - recall: 0.4125
Epoch 2: val_accuracy improved from 0.58209 to 0.58507, saving model to ../models/weights\ResNet152v2_1.00.00_ss.hdf5
119/119 [==============================] - 63s 529ms/step - loss: 0.6663 - accuracy: 0.6966 - precision: 0.8396 - recall: 0.4125 - val_loss: 1.2944 - val_accuracy: 0.5851 - val_precision: 0.5632 - val_recall: 0.6087
Epoch 3/200
119/119 [==============================] - ETA: 0s - 

In [7]:
##
# Clear memories after model training:
#
model_development.clear_memory()